Early regression attempts not used in final modeling process but kept in case I decided to return to regression. 


In [2]:
from sklearn.ensemble import  AdaBoostRegressor
from sklearn.model_selection import train_test_split, cross_val_score
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import RidgeCV, LinearRegression
from sklearn.model_selection import GridSearchCV

%matplotlib inline

In [3]:
#Bring in the data

nba = pd.read_csv('./data/nba_analysis_data.csv')
nba.dropna(inplace = True)

In [4]:
#seperate test from train

nba_train = nba[nba.test == 0]
nba_test = nba[nba.test ==1]

In [5]:
nba_train = nba_train[nba_train.playoff_game == 1]
nba_test = nba_test[nba_test.playoff_game == 1]

In [6]:
X_train = nba_train.drop(columns = ['cover','home_win_margin', 
                        'date', 'dataset','line_cv',
                       'home_starter5', 'teams', 'away_team',
                        'away_starter2', 'away_starter3',
                       'away_starter4', 'away_starter5',
                       'ref_1', 'ref_3', 'crew_referees',
                      'away_pace', 'away_spread',
                        'away_line_cv',  'away_cover',
                       'home_payout', 'away_payout'])

y_train = nba_train['home_win_margin']

In [7]:
X_test= nba_test.drop(columns = ['cover','home_win_margin', 
                        'date', 'dataset','line_cv', 
                       'home_starter5', 'teams', 'away_team',
                       'away_starter2', 'away_starter3',
                       'away_starter4', 'away_starter5',
                       'ref_1', 'ref_3', 'crew_referees',
                      'away_pace', 'away_spread',
                        'away_line_cv',  'away_cover',
                       'home_payout', 'away_payout'])

y_test = nba_test.home_win_margin

In [8]:
print(X_train.shape)
print(X_test.shape)

(711, 152)
(150, 152)


In [9]:

# #create interactions and squared terms
# poly = PolynomialFeatures(degree = 2, interaction_only= True)
# X_poly = poly.fit_transform(X_train)
# X_poly_test = poly.transform(X_test)
# X_poly = pd.DataFrame(X_poly, 
#                       columns = poly.get_feature_names(X_train.columns))
# X_poly_test = pd.DataFrame(X_poly_test, 
#                            columns = poly.get_feature_names(X_test.columns))


In [10]:
#scaling data to use in various other methods
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

/Users/bryancombs/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/bryancombs/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/bryancombs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


In [11]:
from sklearn.decomposition import PCA


In [12]:
pc = PCA(n_components=70)
X_train_pc = pc.fit_transform(X_train_scaled)
X_test_pc = pc.transform(X_test_scaled)



In [13]:

var_exp = pc.explained_variance_ratio_
print('Explained variance:            ', var_exp)

cum_var_exp = np.cumsum(var_exp)
print('Cumulative explained variance: ', cum_var_exp)

Explained variance:             [0.1150814  0.10161556 0.06076771 0.04233699 0.03905918 0.03674313
 0.03382649 0.03089665 0.03011245 0.02776349 0.02619915 0.02474561
 0.0229015  0.02221144 0.01922258 0.01823352 0.01722797 0.01566985
 0.01495293 0.0146337  0.01404968 0.01356968 0.01225612 0.01194559
 0.01152384 0.01111066 0.00990036 0.00976715 0.00948426 0.0090424
 0.00840059 0.00799855 0.0076508  0.0074893  0.0069488  0.00667512
 0.0063872  0.00617169 0.00582923 0.00560947 0.00515205 0.00480785
 0.00460111 0.00454045 0.00437197 0.00403631 0.00382312 0.00365137
 0.00355774 0.00337633 0.00326911 0.00316709 0.00303685 0.00290507
 0.00280467 0.00259775 0.00253597 0.00243553 0.00226    0.00223197
 0.00201974 0.00192338 0.00185071 0.00179651 0.00176979 0.00162358
 0.00153155 0.00142934 0.00135104 0.00131297]
Cumulative explained variance:  [0.1150814  0.21669697 0.27746467 0.31980166 0.35886084 0.39560397
 0.42943046 0.4603271  0.49043955 0.51820303 0.54440219 0.56914779
 0.5920493  0.614260

### Linear Regression

In [14]:
ols = LinearRegression()
ols.fit(X_train_pc, y_train)
ols.score(X_train_pc, y_train)

0.23927730954786852

In [15]:
ols.score(X_test_pc, y_test)

0.057092201861608016

### Boosting

In [16]:
ad = AdaBoostRegressor()
ad_params = { 'n_estimators'      : [300],
             'learning_rate'         : [ .01 ],
             
            }

In [17]:
gs = GridSearchCV(ad, param_grid= ad_params)
gs.fit(X_train_pc, y_train)
print(gs.best_params_)
print(gs.best_score_)

/Users/bryancombs/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


{'learning_rate': 0.01, 'n_estimators': 300}
0.039955207474881904


In [18]:
gs.score(X_test_pc,y_test)

0.0629868363322279

In [19]:
y_hat_ad = gs.predict(X_test_pc)

In [20]:
rr = RidgeCV(alphas = [70000, 90000, 80000])


In [21]:
rr.fit(X_train_pc, y_train);

In [22]:
rr.score(X_train_pc, y_train)

0.020266439526816837

In [23]:
rr.score(X_test_pc, y_test)

0.013912066073947018

In [24]:
y_hat_rr = rr.predict(X_test_pc)

In [25]:
rr.alpha_

70000

### Support Vector Machine

In [26]:
svm = SVR()
svm_params = {'kernel' : ['rbf'],
              'C'      : [.1, .02, .05]}

In [27]:
gs = GridSearchCV(svm, param_grid= svm_params)
gs.fit(X_train_pc, y_train)
print(gs.best_params_)
print(gs.best_score_)

/Users/bryancombs/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/bryancombs/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/bryancombs/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/bryancombs/anaconda3/lib/python3.6/site-packages/sklearn/svm/ba

{'C': 0.1, 'kernel': 'rbf'}
-0.00482538625784712


/Users/bryancombs/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [28]:
gs.score(X_test_pc, y_test)

-0.00867359671236101

In [29]:
y_hat_svm = gs.predict(X_test_pc)


### Neural Network

Going to fit a neural network with the X features.   I am not concerned about inference.  Accurate prediction is all that matters.  So am going to set up an neural network as one of the final predictors of Home Win Margin.

In [30]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

/Users/bryancombs/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [31]:
model = Sequential()
model.add(Dense(100, 
                activation = 'relu',
                input_dim = X_train_pc.shape[1]))
#model.add(Dense(18, activation = 'relu'))

model.add(Dense(1, activation = None)) #output layer 

In [32]:
model.compile(loss = 'mean_squared_error', optimizer='adam' )
model.fit(X_train_pc, 
          y_train, epochs =18, 
          batch_size= 64, 
          validation_data=(X_test_pc, y_test))

Train on 711 samples, validate on 150 samples
Epoch 1/18
711/711 [==============================] - 0s 266us/step - loss: 201.0874 - val_loss: 249.4476
Epoch 2/18
711/711 [==============================] - 0s 25us/step - loss: 190.9766 - val_loss: 244.5769
Epoch 3/18
711/711 [==============================] - 0s 25us/step - loss: 183.3407 - val_loss: 240.2046
Epoch 4/18
711/711 [==============================] - 0s 24us/step - loss: 176.4535 - val_loss: 237.2074
Epoch 5/18
711/711 [==============================] - 0s 26us/step - loss: 170.6207 - val_loss: 234.3651
Epoch 6/18
711/711 [==============================] - 0s 24us/step - loss: 165.6940 - val_loss: 232.0352
Epoch 7/18
711/711 [==============================] - 0s 25us/step - loss: 160.9806 - val_loss: 229.4488
Epoch 8/18
711/711 [==============================] - 0s 23us/step - loss: 157.0058 - val_loss: 228.0151
Epoch 9/18
711/711 [==============================] - 0s 25us/step - loss: 153.2476 - val_loss: 226.1125
Epoch 10

In [33]:
y_hat_nn = model.predict(X_test_pc)
predictions = pd.DataFrame(y_hat_nn, columns= ['y_hat_nn'])
predictions

,y_hat_nn
0,6.721441
1,10.446864
2,13.049931
3,3.719043
4,2.978666
5,0.601872
6,1.756836
7,0.202521
8,1.684897
9,9.276322


In [34]:
predictions['y_hat_rr'] = y_hat_rr
predictions['y_hat_svm'] = y_hat_svm
predictions['y_hat_ad'] = y_hat_ad


In [35]:
predictions.head(10)

,y_hat_nn,y_hat_rr,y_hat_svm,y_hat_ad
0,6.721441,4.975133,5.722074,-0.557143
1,10.446864,4.955814,5.713785,3.569307
2,13.049931,5.075065,5.744758,4.531120
3,3.719043,4.576165,5.575304,5.844828
4,2.978666,4.090690,5.503234,3.574297
5,0.601872,3.993053,5.450361,4.956395
6,1.756836,4.233513,5.462331,3.720524
7,0.202521,4.489014,5.450530,5.107206
8,1.684897,4.777633,5.533584,4.740458
9,9.276322,5.177623,5.659446,6.406654


In [36]:
y_s = pd.DataFrame(y_test)
y_s.reset_index(inplace = True)
predictions['y_test'] = y_s['home_win_margin']
predictions.head()

,y_hat_nn,y_hat_rr,y_hat_svm,y_hat_ad,y_test
0,6.721441,4.975133,5.722074,-0.557143,7.0
1,10.446864,4.955814,5.713785,3.569307,8.0
2,13.049931,5.075065,5.744758,4.531120,4.0
3,3.719043,4.576165,5.575304,5.844828,-17.0
4,2.978666,4.090690,5.503234,3.574297,-9.0


In [37]:
X_test.reset_index(inplace = True)
predictions['spread'] = X_test['spread']
predictions

,y_hat_nn,y_hat_rr,y_hat_svm,y_hat_ad,y_test,spread
0,6.721441,4.975133,5.722074,-0.557143,7.0,-5.0
1,10.446864,4.955814,5.713785,3.569307,8.0,-5.5
2,13.049931,5.075065,5.744758,4.531120,4.0,-5.5
3,3.719043,4.576165,5.575304,5.844828,-17.0,-1.5
4,2.978666,4.090690,5.503234,3.574297,-9.0,2.0
5,0.601872,3.993053,5.450361,4.956395,-22.0,0.0
6,1.756836,4.233513,5.462331,3.720524,27.0,-5.0
7,0.202521,4.489014,5.450530,5.107206,19.0,-4.5
8,1.684897,4.777633,5.533584,4.740458,1.0,-5.5
9,9.276322,5.177623,5.659446,6.406654,-3.0,-5.5


In [38]:
predictions['y_hat_ave'] = (predictions.y_hat_nn + 
                           
                            predictions.y_hat_rr + 
                            predictions.y_hat_nn)/3

In [39]:
predictions.head(20)

,y_hat_nn,y_hat_rr,y_hat_svm,y_hat_ad,y_test,spread,y_hat_ave
0,6.721441,4.975133,5.722074,-0.557143,7.0,-5.0,6.139339
1,10.446864,4.955814,5.713785,3.569307,8.0,-5.5,8.616514
2,13.049931,5.075065,5.744758,4.531120,4.0,-5.5,10.391642
3,3.719043,4.576165,5.575304,5.844828,-17.0,-1.5,4.004750
4,2.978666,4.090690,5.503234,3.574297,-9.0,2.0,3.349341
5,0.601872,3.993053,5.450361,4.956395,-22.0,0.0,1.732266
6,1.756836,4.233513,5.462331,3.720524,27.0,-5.0,2.582395
7,0.202521,4.489014,5.450530,5.107206,19.0,-4.5,1.631352
8,1.684897,4.777633,5.533584,4.740458,1.0,-5.5,2.715809
9,9.276322,5.177623,5.659446,6.406654,-3.0,-5.5,7.910089


In [40]:
predictions['predict_svm'] =  (predictions.y_hat_svm + predictions.spread).map(
    lambda x: 0 if x <0 else 1)

In [41]:
predictions['predict_ad'] =  (predictions.y_hat_ad + predictions.spread).map(
    lambda x: 0 if x <0 else 1)

In [42]:
predictions['predict_rr'] =  (predictions.y_hat_rr + predictions.spread).map(
    lambda x: 0 if x <0 else 1)

In [43]:
predictions['predict_nn'] =  (predictions.y_hat_nn + predictions.spread).map(
    lambda x: 0 if x <0 else 1)

In [44]:
predictions['actual'] = (predictions.y_test + predictions.spread).map(lambda x: 0 if x< 0 else 1)

In [45]:
predictions['predict'] = (predictions['predict_svm']+ 
    predictions['predict_rr'] + predictions['predict_nn'])

In [46]:
predictions.head()

,y_hat_nn,y_hat_rr,y_hat_svm,y_hat_ad,y_test,spread,y_hat_ave,predict_svm,predict_ad,predict_rr,predict_nn,actual,predict
0,6.721441,4.975133,5.722074,-0.557143,7.0,-5.0,6.139339,1,0,0,1,1,2
1,10.446864,4.955814,5.713785,3.569307,8.0,-5.5,8.616514,1,0,0,1,1,2
2,13.049931,5.075065,5.744758,4.531120,4.0,-5.5,10.391642,1,0,0,1,0,2
3,3.719043,4.576165,5.575304,5.844828,-17.0,-1.5,4.004750,1,1,1,1,0,3
4,2.978666,4.090690,5.503234,3.574297,-9.0,2.0,3.349341,1,1,1,1,0,3


In [52]:
print(classification_report(predictions.actual, predictions.predict_svc))

AttributeError: 'DataFrame' object has no attribute 'predict_svc'

In [48]:
confusion_matrix(predictions.actual, predictions.predict_svm)

array([[29, 48],
       [32, 41]])

In [49]:
predictions.predict.value_counts(normalize = True)

3    0.406667
1    0.280000
0    0.160000
2    0.153333
Name: predict, dtype: float64

In [50]:
predictions.groupby(['predict'])['actual'].mean()

predict
0    0.416667
1    0.595238
2    0.521739
3    0.426230
Name: actual, dtype: float64

In [51]:
predictions[predictions.predict == 0].away_payout.sum()

AttributeError: 'DataFrame' object has no attribute 'away_payout'